# DX 704 Week 4 Project

This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

In [20]:
import numpy as np
import pandas as pd

In [21]:
recipes = pd.read_csv('https://raw.githubusercontent.com/bu-cds-dx704/dx704-project-04/refs/heads/main/recipes.tsv', sep="\t")
recipes = recipes.set_index("recipe_slug")
recipes

,recipe_title,recipe_introduction
recipe_slug,,
falafel,Falafel,Falafel is a popular Middle Eastern dish made ...
spamburger,Spamburger,Spamburger is a type of hamburger that is made...
bacon-fried-rice,Bacon Fried Rice,Bacon fried rice is a savory and satisfying di...
chicken-fingers,Chicken Fingers,Chicken fingers are a popular dish made from c...
apple-crisp,Apple Crisp,Apple crisp is a classic dessert made with bak...
...,...,...
bacon-mac-and-cheese,Bacon Mac And Cheese,Bacon mac and cheese is a delicious and comfor...
chicken-alfredo-lasagna,Chicken Alfredo Lasagna,Chicken alfredo lasagna is a delicious twist o...
classic-beef-lasagna,Classic Beef Lasagna,Classic beef lasagna is a hearty and comfortin...


In [22]:
def add_feature(feature_name):
    recipes[feature_name] = recipes["recipe_title"].str.lower().str.contains(feature_name.lower())

In [23]:
add_feature("apple")
add_feature("bacon")
add_feature("chicken")
add_feature("crisp")
add_feature("lasagna")
add_feature("veg")

recipes

,recipe_title,recipe_introduction,apple,bacon,chicken,crisp,lasagna,veg
recipe_slug,,,,,,,,
falafel,Falafel,Falafel is a popular Middle Eastern dish made ...,False,False,False,False,False,False
spamburger,Spamburger,Spamburger is a type of hamburger that is made...,False,False,False,False,False,False
bacon-fried-rice,Bacon Fried Rice,Bacon fried rice is a savory and satisfying di...,False,True,False,False,False,False
chicken-fingers,Chicken Fingers,Chicken fingers are a popular dish made from c...,False,False,True,False,False,False
apple-crisp,Apple Crisp,Apple crisp is a classic dessert made with bak...,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...
bacon-mac-and-cheese,Bacon Mac And Cheese,Bacon mac and cheese is a delicious and comfor...,False,True,False,False,False,False
chicken-alfredo-lasagna,Chicken Alfredo Lasagna,Chicken alfredo lasagna is a delicious twist o...,False,False,True,False,True,False
classic-beef-lasagna,Classic Beef Lasagna,Classic beef lasagna is a hearty and comfortin...,False,False,False,False,True,False


In [24]:
ratings = pd.DataFrame({"recipe_title": recipes["recipe_title"],
                               "rating_bad": np.random.uniform(low=2, high=3, size=len(recipes))},
                              index=recipes.index)
ratings.loc[recipes['veg'], 'rating_bad'] = 1
ratings.loc[recipes['bacon'], 'rating_bad'] = 5
ratings['rating'] = ratings['rating_bad']
ratings

,recipe_title,rating_bad,rating
recipe_slug,,,
falafel,Falafel,2.299072,2.299072
spamburger,Spamburger,2.797707,2.797707
bacon-fried-rice,Bacon Fried Rice,5.000000,5.000000
chicken-fingers,Chicken Fingers,2.739505,2.739505
apple-crisp,Apple Crisp,2.597907,2.597907
...,...,...,...
bacon-mac-and-cheese,Bacon Mac And Cheese,5.000000,5.000000
chicken-alfredo-lasagna,Chicken Alfredo Lasagna,2.452441,2.452441
classic-beef-lasagna,Classic Beef Lasagna,2.487656,2.487656


In [25]:
ratings["rating"] = (ratings["rating_bad"] - 1) * 0.25

In [32]:
# Force 10 lowest ratings to 0 and 10 highest to 1
lowest_idx = ratings["rating"].nsmallest(10).index
ratings.loc[lowest_idx, "rating"] = 0.0

highest_idx = ratings["rating"].nlargest(10).index
ratings.loc[highest_idx, "rating"] = 1.0


In [33]:
ratings

,recipe_title,rating_bad,rating
recipe_slug,,,
falafel,Falafel,2.299072,0.324768
spamburger,Spamburger,2.797707,0.449427
bacon-fried-rice,Bacon Fried Rice,5.000000,1.000000
chicken-fingers,Chicken Fingers,2.739505,0.434876
apple-crisp,Apple Crisp,2.597907,0.399477
...,...,...,...
bacon-mac-and-cheese,Bacon Mac And Cheese,5.000000,1.000000
chicken-alfredo-lasagna,Chicken Alfredo Lasagna,2.452441,0.363110
classic-beef-lasagna,Classic Beef Lasagna,2.487656,0.371914


In [34]:
ratings.reset_index().to_csv("ratings.tsv", sep="\t", index=False)

Submit "ratings.tsv" in Gradescope.

## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

In [ ]:
ratings = pd.read_csv("ratings.tsv", sep="\t")

tags = pd.read_csv("recipe-tags.tsv", sep="\t")   

features = pd.crosstab(tags["recipe_slug"], tags["recipe_tag"]).astype(np.uint8).reset_index()

features = (
    features.set_index("recipe_slug")
              .reindex(ratings["recipe_slug"], fill_value=0)
              .reset_index()
)

features

recipe_tag,recipe_slug,alfredo,almond,american,appetizer,appetizers,apple,asiancuisine,asparagus,avocado,...,udonnoodles,vanilla,vanillaicecream,vegan,vegetables,vegetarian,warm,whippedcream,winter,yeastdough
0,falafel,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,spamburger,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,bacon-fried-rice,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,chicken-fingers,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,apple-crisp,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,bacon-mac-and-cheese,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,chicken-alfredo-lasagna,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,classic-beef-lasagna,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,vegetarian-mushroom-lasagna,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [49]:
# Add bias column
features.insert(1, "bias", 1)
features

recipe_tag,recipe_slug,bias,alfredo,almond,american,appetizer,appetizers,apple,asiancuisine,asparagus,...,udonnoodles,vanilla,vanillaicecream,vegan,vegetables,vegetarian,warm,whippedcream,winter,yeastdough
0,falafel,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,spamburger,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,bacon-fried-rice,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,chicken-fingers,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,apple-crisp,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,bacon-mac-and-cheese,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,chicken-alfredo-lasagna,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,classic-beef-lasagna,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,vegetarian-mushroom-lasagna,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [50]:
features.to_csv("features.tsv", sep="\t", index=False)

Submit "features.tsv" in Gradescope.

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

In [52]:
# YOUR CHANGES HERE
from sklearn.linear_model import Ridge

# Load data in
ratings = pd.read_csv("ratings.tsv", sep="\t")
features = pd.read_csv("features.tsv", sep="\t")

X = features.drop(columns=["recipe_slug"]).select_dtypes(include=[np.number])
y = ratings["rating"].astype(float).values

# Fit ridge regression
ridge = Ridge(alpha=1, fit_intercept=False)
ridge.fit(X, y)

model = pd.DataFrame({
    "feature": X.columns,
    "coefficient": ridge.coef_
})

model

,feature,coefficient
0,bias,0.335223
1,alfredo,0.023080
2,almond,0.037976
3,american,0.002850
4,appetizer,0.073383
...,...,...
292,vegetarian,-0.043215
293,warm,-0.015089
294,whippedcream,0.002056
295,winter,0.003708


Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not include the bias.

In [53]:
# YOUR CHANGES HERE
model.rename(columns={"feature": "recipe_tag"}, inplace=True)
model.to_csv("model.tsv", sep="\t", index=False)

Submit "model.tsv" in Gradescope.

## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [54]:
ratings  = pd.read_csv("ratings.tsv",  sep="\t")        
features = pd.read_csv("features.tsv", sep="\t")        
model    = pd.read_csv("model.tsv",    sep="\t")       

coef = model.set_index("recipe_tag")["coefficient"]

use_cols = [c for c in features.columns if c in coef.index]
missing  = [t for t in coef.index if t not in features.columns]
for t in missing:
    features[t] = 0
    use_cols.append(t)

use_cols = list(dict.fromkeys(use_cols))
X    = features[use_cols]
beta = coef.loc[use_cols].values
score_estimate = X.dot(beta)

id_candidates = ["recipe_slug","slug","recipe_id","id"]
id_col = next((c for c in id_candidates if c in features.columns), None)

estimates = pd.DataFrame({"score_estimate": score_estimate})
if id_col:
    estimates.insert(0, id_col, features[id_col].values)
else:
    estimates.reset_index(names="row_index", inplace=True)

estimates

,recipe_slug,score_estimate
0,falafel,0.335249
1,spamburger,0.451271
2,bacon-fried-rice,0.992189
3,chicken-fingers,0.437440
4,apple-crisp,0.394528
...,...,...
95,bacon-mac-and-cheese,0.964565
96,chicken-alfredo-lasagna,0.340030
97,classic-beef-lasagna,0.337526
98,vegetarian-mushroom-lasagna,0.081697


In [55]:
estimates.to_csv("estimates.tsv", sep="\t", index=False)

Submit "estimates.tsv" in Gradescope.

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [65]:
def calculate_bounds(recipe_choices=None, alpha=2.0):
    if "recipe_slug" not in features.columns:
        raise KeyError("features must have a 'recipe_slug' column.")
    if "recipe_slug" not in ratings.columns or "rating" not in ratings.columns:
        raise KeyError("ratings must have 'recipe_slug' and 'rating' columns.")

    X = (
        features.drop(columns=["recipe_slug"])
                .select_dtypes(include=[np.number])
                .astype("float64")
    )
    if X.shape[1] == 0:
        raise ValueError("No numeric feature columns.")

    default_choices = ratings["recipe_slug"].tolist()
    if recipe_choices is None:
        recipe_choices = default_choices
    else:
        recipe_choices = list(recipe_choices)

    idx = pd.Index(features["recipe_slug"])
    row_idx = idx.get_indexer(recipe_choices)
    if (row_idx < 0).any():
        missing = [recipe_choices[i] for i, r in enumerate(row_idx) if r < 0]
        raise KeyError(f"Missing in features: {missing}")

    D = X.iloc[row_idx].to_numpy()
    c = (
        ratings.set_index("recipe_slug")
               .reindex(recipe_choices)["rating"]
               .to_numpy(dtype="float64")
    )

    d = X.shape[1]
    A = np.eye(d) + D.T @ D
    theta = np.linalg.solve(A, D.T @ c)

    Xall = X.to_numpy()
    means = Xall @ theta
    A_inv = np.linalg.inv(A)
    vars_ = np.einsum("ij,jk,ik->i", Xall, A_inv, Xall)
    bounds = means + alpha * np.sqrt(np.maximum(vars_, 0.0))

    out = pd.DataFrame(
        {"score_estimate": means, "score_bound": bounds},
        index=features["recipe_slug"],
    )
    
    return out

In [66]:
calculate_bounds(alpha=2).sort_values("score_bound", ascending=False)

,score_estimate,score_bound
recipe_slug,,
bacon-fried-rice,0.992189,2.858345
bacon-wrapped-chicken,0.963715,2.803192
bacon-wrapped-shrimp-skewers,0.970057,2.795346
bacon-mac-and-cheese,0.964565,2.786119
bacon-wrapped-asparagus,0.972120,2.784369
...,...,...
blueberry-crisp,0.095553,1.684581
vegetarian-mushroom-lasagna,0.081697,1.671818
spinach-and-ricotta-stuffed-shells,0.077596,1.632613


In [67]:
bounds = calculate_bounds(alpha=2).reset_index()[["recipe_slug", "score_bound"]]
bounds.to_csv("bounds.tsv", sep="\t", index=False)

Submit "bounds.tsv" in Gradescope.

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

In [69]:
def make_recommendations(n=100, alpha=2.0):
    r_lookup = ratings.set_index("recipe_slug")["rating"]

    chosen = []
    recs = []

    for _ in range(min(n, len(features))):
        bounds = calculate_bounds(recipe_choices=chosen, alpha=alpha)
        bounds = bounds.loc[~bounds.index.isin(chosen)]

        if bounds.empty:
            break

        best_slug = bounds["score_bound"].idxmax()
        best_bound = float(bounds.loc[best_slug, "score_bound"])
        reward = float(r_lookup.loc[best_slug])

        recs.append({"recipe_slug": best_slug, "score_bound": best_bound, "reward": reward})
        chosen.append(best_slug)

    recs_df = pd.DataFrame(recs, columns=["recipe_slug", "score_bound", "reward"])
    return recs_df

In [71]:
recommendations = make_recommendations(n=100, alpha=2.0)
recommendations

,recipe_slug,score_bound,reward
0,apple-crumble,7.483315,0.412042
1,ma-la-chicken,7.220058,0.476042
2,quesadillas,7.207316,0.387201
3,ramen,7.186941,0.000000
4,chocolate-babka,6.909184,0.481346
...,...,...,...
95,spinach-and-ricotta-stuffed-shells,2.905990,0.000000
96,chocolate-souffle,2.819209,0.311012
97,butter-croissants,2.594350,0.297473
98,berry-crumble,2.528193,0.000000


In [72]:
recommendations.to_csv("recommendations.tsv", sep="\t", index=False)

Submit "recommendations.tsv" in Gradescope.

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.


Submit "project.ipynb" in Gradescope.